In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import random
from pprint import pprint

In [2]:
import_source="https://raw.githubusercontent.com/SimbarasheJehol/MachineLearningProject/master/dataProccesed2.csv"
df = pd.read_csv(import_source)
print("data is loaded")

data is loaded


In [3]:
df=df.drop('Unnamed: 0',axis=1)

In [4]:
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education_num',
       'martial_status', 'occupation', 'RelationShip', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'age-binned', 'fnlwgt-binned', 'education_num-binned',
       'capital_gain-binned', 'capital_loss-binned', 'hours_per_week-binned',
       'education_num_rand', 'income_class'],
      dtype='object')

In [5]:
df=df.drop( "fnlwgt", axis=1)\
                .drop( "capital_gain",axis=1)\
                .drop("capital_loss",axis=1)\
                .drop("education_num_rand",axis=1)\
                .drop( "fnlwgt-binned", axis=1)\
                .drop( "capital_gain-binned",axis=1)\
                .drop("capital_loss-binned",axis=1)\
                .drop("education_num-binned",axis=1)\
                .drop("hours_per_week-binned",axis=1)\
               

In [6]:
##target_variable=df["income_class"]
df.shape

(32561, 13)

In [7]:
def train_test_split(df, test_size):
    
    if isinstance(test_size, float):
        test_size = round(test_size * len(df))

    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df

In [31]:
#random.seed(0)// this will always return the same test_df dataframe
train_df, test_df = train_test_split(df, test_size=0.2)

In [35]:
test_df.sample(5)


,age,workclass,education,education_num,martial_status,occupation,RelationShip,race,sex,hours_per_week,native_country,age-binned,income_class
14875,25,Federal-gov,Some-college,10,Married-civ-spouse,Protective-serv,Husband,Black,Male,40,United-States,starter_workclass,<=50K
22773,41,Self-emp-inc,9th,5,Married-civ-spouse,Sales,Wife,White,Female,70,Dominican-Republic,mid_workclass,<=50K
15458,56,Private,9th,5,Divorced,Adm-clerical,Not-in-family,White,Male,40,United-States,Advanced_workclass,<=50K
29634,62,Private,HS-grad,9,Never-married,Adm-clerical,Not-in-family,White,Female,32,United-States,diminishing_workclass,<=50K
9152,32,State-gov,HS-grad,9,Married-civ-spouse,Protective-serv,Husband,White,Male,40,United-States,junior work_class,<=50K


In [33]:
data = train_df.values ##speeding up the code
data[:5]

array([[39, ' State-gov', ' Bachelors', 13, ' Never-married',
        ' Adm-clerical', ' Not-in-family', ' White', ' Male', 40,
        ' United-States', 'mid_workclass', ' <=50K'],
       [50, ' Self-emp-not-inc', ' Bachelors', 13, ' Married-civ-spouse',
        ' Exec-managerial', ' Husband', ' White', ' Male', 13,
        ' United-States', 'Advanced_workclass', ' <=50K'],
       [38, ' Private', ' HS-grad', 9, ' Divorced', ' Handlers-cleaners',
        ' Not-in-family', ' White', ' Male', 40, ' United-States',
        'mid_workclass', ' <=50K'],
       [28, ' Private', ' Bachelors', 13, ' Married-civ-spouse',
        ' Prof-specialty', ' Wife', ' Black', ' Female', 40, ' Cuba',
        'junior work_class', ' <=50K'],
       [37, ' Private', ' Masters', 14, ' Married-civ-spouse',
        ' Exec-managerial', ' Wife', ' White', ' Female', 40,
        ' United-States', 'mid_workclass', ' <=50K']], dtype=object)

In [11]:
def check_purity(data):
    
    label_column = data[:,-1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

In [12]:
check_purity(train_df.values)

False

In [13]:
def classify_data(data):
    
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

In [14]:
def get_potential_splits(data):
    
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):          # excluding the last column which is the label
        values = data[:, column_index]
        unique_values = np.unique(values)
        
        type_of_feature = FEATURE_TYPES[column_index]
        if type_of_feature == "continuous":
            potential_splits[column_index] = []
            for index in range(len(unique_values)):
                if index != 0:
                    current_value = unique_values[index]
                    previous_value = unique_values[index - 1]
                    potential_split = (current_value + previous_value) / 2

                    potential_splits[column_index].append(potential_split)
        
        # feature is categorical
        # (there need to be at least 2 unique values, otherwise in the
        # split_data function data_below would contain all data points
        # and data_above would be empty)
        elif len(unique_values) > 1:
            potential_splits[column_index] = unique_values
    
    return potential_splits

In [15]:
def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]

    type_of_feature = FEATURE_TYPES[split_column]
    if type_of_feature == "continuous":
        data_below = data[split_column_values <= split_value]
        data_above = data[split_column_values >  split_value]
    
    # feature is categorical   
    else:
        data_below = data[split_column_values == split_value]
        data_above = data[split_column_values != split_value]
    
    return data_below, data_above

In [16]:
def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

In [17]:
def calculate_overall_entropy(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

In [18]:
def determine_best_split(data, potential_splits):
    
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

In [19]:
def determine_type_of_feature(df):
    
    feature_types = []
    n_unique_values_treshold = 20
    for feature in df.columns:
        if feature != "income_class":
            unique_values = df[feature].unique()
            example_value = unique_values[0]

            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_treshold):
                feature_types.append("categorical")
            else:
                feature_types.append("continuous")
    
    return feature_types

In [20]:
def decision_tree_algorithm(df, counter=0, min_samples=2, max_depth=5):
    
    # data preparations
    if counter == 0:
        global COLUMN_HEADERS, FEATURE_TYPES
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = determine_type_of_feature(df)
        data = df.values
    else:
        data = df           
    
    
    # base cases
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        # determine question
        feature_name = COLUMN_HEADERS[split_column]
        type_of_feature = FEATURE_TYPES[split_column]
        if type_of_feature == "continuous":
            question = "{} <= {}".format(feature_name, split_value)
            
        # feature is categorical
        else:
            question = "{} = {}".format(feature_name, split_value)
        
        # instantiate sub-tree
        sub_tree = {question: []}
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth)
        
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [21]:
tree = decision_tree_algorithm(train_df, max_depth=3)
pprint(tree)

{'martial_status =  Married-civ-spouse': [{'education_num = 13': [' >50K',
                                                                  {'occupation =  Prof-specialty': [' '
                                                                                                    '>50K',
                                                                                                    ' '
                                                                                                    '<=50K']}]},
                                          ' <=50K']}


In [22]:
example = test_df.iloc[:,:]
example

,age,workclass,education,education_num,martial_status,occupation,RelationShip,race,sex,hours_per_week,native_country,age-binned,income_class
17956,36,Private,HS-grad,9,Separated,Sales,Unmarried,Black,Female,17,United-States,mid_workclass,<=50K
17785,42,Private,HS-grad,9,Divorced,Handlers-cleaners,Unmarried,Black,Female,40,United-States,senior_workclass,<=50K
3339,31,Private,HS-grad,9,Widowed,Craft-repair,Unmarried,White,Female,40,United-States,junior work_class,<=50K
5337,47,Private,Masters,14,Divorced,Prof-specialty,Unmarried,White,Female,50,United-States,senior_workclass,>50K
18035,27,Private,HS-grad,9,Never-married,Transport-moving,Not-in-family,White,Male,70,United-States,junior work_class,<=50K
13100,50,Private,Some-college,10,Married-civ-spouse,Sales,Husband,White,Male,40,United-States,Advanced_workclass,<=50K
6032,34,Private,Assoc-acdm,12,Never-married,Adm-clerical,Not-in-family,Asian-Pac-Islander,Female,60,United-States,mid_workclass,>50K
8203,20,Private,Some-college,10,Never-married,Priv-house-serv,Own-child,White,Female,20,United-States,starter_workclass,<=50K
6864,54,Private,Preschool,1,Married-civ-spouse,Farming-fishing,Husband,White,Male,60,United-States,Advanced_workclass,<=50K
32554,53,Private,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,40,United-States,Advanced_workclass,>50K


In [23]:
def classify_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" "),question.split(" "),question.split(" ")

    # ask question
    if comparison_operator == "<=":
        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    
    # feature is categorical
    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)

In [24]:
def calculate_accuracy(df, tree):

    df["classification"] = df.apply(classify_example, args=(tree,), axis=1)
    df["classification_correct"] = df["classification"] == df["income_class"]
    
    accuracy = df["classification_correct"].mean()
    
    return accuracy

In [25]:
calculate_accuracy(test_df,tree)

/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:851: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


0.761517199017199

In [26]:
train_df, test_df = train_test_split(df, test_size=0.3)
tree = decision_tree_algorithm(train_df)
accuracy = calculate_accuracy(test_df, tree)

In [27]:
print(tree)

{'martial_status =  Married-civ-spouse': [{'education_num = 13': [{'hours_per_week <= 31.0': [{'RelationShip =  Wife': [' >50K', ' <=50K']}, {'age-binned = starter_workclass': [' <=50K', ' >50K']}]}, {'occupation =  Prof-specialty': [{'hours_per_week <= 26.5': [{'sex =  Male': [' <=50K', ' >50K']}, ' >50K']}, {'occupation =  Exec-managerial': [' >50K', ' <=50K']}]}]}, {'age <= 28.5': [{'hours_per_week <= 37.5': [{'workclass =  Self-emp-not-inc': [{'occupation =  Exec-managerial': [' >50K', ' <=50K']}, ' <=50K']}, {'hours_per_week <= 49.5': [{'martial_status =  Married-AF-spouse': [' >50K', ' <=50K']}, ' <=50K']}]}, {'hours_per_week <= 41.5': [{'occupation =  Prof-specialty': [{'education_num = 15': [' >50K', ' <=50K']}, ' <=50K']}, ' <=50K']}]}]}


In [28]:
print(accuracy)

0.762080262080262


In [29]:
test_df


,age,workclass,education,education_num,martial_status,occupation,RelationShip,race,sex,hours_per_week,native_country,age-binned,income_class,classification,classification_correct
10182,39,Private,Some-college,10,Married-civ-spouse,Sales,Husband,White,Male,26,United-States,mid_workclass,<=50K,<=50K,True
16342,53,Private,Some-college,10,Married-civ-spouse,Exec-managerial,Wife,White,Female,40,United-States,Advanced_workclass,>50K,<=50K,False
26843,47,Private,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,45,United-States,senior_workclass,>50K,<=50K,False
18097,72,Self-emp-not-inc,HS-grad,9,Married-civ-spouse,Sales,Husband,White,Male,30,United-States,junior_retired,<=50K,<=50K,True
17207,50,Private,Some-college,10,Married-civ-spouse,Sales,Husband,Asian-Pac-Islander,Male,50,United-States,Advanced_workclass,>50K,<=50K,False
23782,38,Private,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,40,United-States,mid_workclass,>50K,<=50K,False
21401,38,Self-emp-inc,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,70,United-States,mid_workclass,>50K,<=50K,False
24742,24,State-gov,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,White,Male,49,United-States,starter_workclass,>50K,<=50K,False
16117,62,Private,Some-college,10,Married-civ-spouse,Prof-specialty,Husband,Black,Male,35,United-States,diminishing_workclass,>50K,<=50K,False
14358,34,Private,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,35,United-States,mid_workclass,<=50K,<=50K,True


In [30]:
pip install pydotplus


The following command must be run outside of the IPython shell:

    $ pip install pydotplus

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/
